
# Getting started

Once you've chosen your scenario, download the data from the Iowa website in csv format. Start by loading the data with pandas. You may need to parse the date columns appropriately.

In [18]:
import pandas as pd

## Load the data into a DataFrame
liquor = pd.read_csv('../../../../Iowa_Liquor_sales_sample_10pct.csv')

In [21]:
## Make necessary column modifications

## Transform the dates
liquor["Date"] = pd.to_datetime(liquor["Date"], format="%m/%d/%Y")

## Reformat column names to replace spaces with underscores
liquor.columns = [c.replace(' ', '_') for c in liquor.columns]
liquor.columns = [c.replace(')', '') for c in liquor.columns]
liquor.columns = [c.replace('(', '') for c in liquor.columns]

In [19]:
def eda(dataframe):
    
    print "missing values \n", dataframe.isnull().sum()## count number of null values per column
    print ''
    print "dataframe types \n", dataframe.dtypes       ## list data type of each column
    print ''
    print "dataframe shape \n", dataframe.shape        ## rows by columns
    print ''
    print "dataframe describe \n", dataframe.describe()## stats -- mean, min, max, etc.. 
    print ''
    for item in dataframe:                             ## count number of unique values per column
        print item
        print dataframe[item].nunique()

eda(liquor)

missing values 
Date                        0
Store Number                0
City                        0
Zip Code                    0
County Number            1077
County                   1077
Category                   68
Category Name             632
Vendor Number               0
Item Number                 0
Item Description            0
Bottle Volume (ml)          0
State Bottle Cost           0
State Bottle Retail         0
Bottles Sold                0
Sale (Dollars)              0
Volume Sold (Liters)        0
Volume Sold (Gallons)       0
dtype: int64

dataframe types 
Date                      object
Store Number               int64
City                      object
Zip Code                  object
County Number            float64
County                    object
Category                 float64
Category Name             object
Vendor Number              int64
Item Number                int64
Item Description          object
Bottle Volume (ml)         int64
State Bottle Cost

In [23]:
print liquor.Category.value_counts()
print liquor.Category_Name.value_counts()

1031080.0    35373
1012100.0    27087
1011200.0    15342
1062310.0    14631
1031200.0    14001
1022100.0    12109
1011100.0    11547
1081600.0    10902
1032080.0    10668
1062200.0    10062
1062300.0     7282
1011300.0     7081
1071100.0     6929
1051010.0     6589
1041100.0     6559
1032200.0     6506
1082900.0     6299
1081200.0     6284
1012200.0     5375
1052010.0     4614
1081390.0     4249
1081900.0     3394
1031100.0     3304
1081300.0     3221
1042100.0     2939
1012300.0     2791
1012210.0     2288
1081030.0     1914
1081330.0     1650
1051110.0     1580
             ...  
1041150.0      250
1081365.0      235
1052100.0      224
1081230.0      201
1011400.0      190
1041200.0      169
1081220.0      157
1051150.0      144
1032100.0      114
1081350.0       99
1081500.0       86
1082100.0       83
1031090.0       78
1022200.0       63
1081250.0       61
1081210.0       52
1062500.0       38
1081020.0       35
1012400.0       23
1031110.0       15
1091100.0       15
1082200.0   

# Explore the data

Perform some exploratory statistical analysis and make some plots, such as histograms of transaction totals, bottles sold, etc.

In [16]:
import seaborn as sns
import matplotlib.pyplot as plt





# Record your findings

Be sure to write out anything observations from your exploratory analysis.


# Mine the data

Now you are ready to compute the variables you will use for your regression from the data. For example, you may want to compute total sales per store from Jan to March of 2015, mean price per bottle, etc. Refer to the readme for more ideas appropriate to your scenario.

Pandas is your friend for this task. Take a look at the operations here for ideas on how to make the best use of pandas and feel free to search for blog and Stack Overflow posts to help you group data by certain variables and compute sums, means, etc. You may find it useful to create a new data frame to house this summary data.
 


# Refine the data

Look for any statistical relationships, correlations, or other relevant properties of the dataset.

# Build your models

Using scikit-learn or statsmodels, build the necessary models for your scenario. Evaluate model fit.

# Plot your results

Again make sure that you record any valuable information. For example, in the tax scenario, did you find the sales from the first three months of the year to be a good predictor of the total sales for the year? Plot the predictions versus the true values and discuss the successes and limitations of your models

# Present the Results

Present your conclusions and results. If you have more than one interesting model feel free to include more than one along with a discussion. Use your work in this notebook to prepare your write-up.